In [30]:
from supabase import create_client
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()
supabase = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))

def upsert_user_memory(memory_data: dict):
    memory_data["created_at"] = datetime.utcnow().isoformat()
    return supabase.table("User_Profiles").upsert(memory_data).execute()

In [31]:
def get_user_memory(user_id: int) -> dict:
    res = supabase.table("User_Profiles").select("*").eq("id", user_id).execute()
    return res.data[0] if res.data else {}

In [32]:
def update_user_fields(user_id: int, fields: dict):
    fields["created_at"] = datetime.utcnow().isoformat()
    return supabase.table("User_Profiles").update(fields).eq("id", user_id).execute()

In [33]:
update_user_fields(123, {"likes": ["vanilla", "hazelnut"]})

APIResponse[~_ReturnT](data=[{'id': 123, 'created_at': '2025-07-05T10:48:49.547116+00:00', 'name': 'Paddy', 'location': 'Pune', 'likes': ['vanilla', 'hazelnut'], 'dislikes': ['too much milk'], 'allergies': ['nuts']}], count=None)

In [34]:
def delete_user_memory(user_id:int):
    return supabase.table("User_Profiles").delete().eq("id", user_id).execute()

In [35]:
user_id = 234  # use an existing or new UUID

# Upsert memory
memory_data = {
    "id": user_id,
    "name": "Pandhari",
    "likes": ["strong coffee"],
    "allergies": ["nuts"],
    "location": "Parbhani",
}
print("Upserting...")
print(upsert_user_memory(memory_data))

# Get memory
print("\nFetching...")
print(get_user_memory(user_id))

# Update specific field
print("\nUpdating allergies...")
print(update_user_fields(user_id, {"allergies": ["dairy", "nuts"]}))

# Delete memory
# print("\nDeleting...")
# print(delete_user_memory(user_id))

Upserting...
data=[{'id': 234, 'created_at': '2025-07-05T10:48:53.262985+00:00', 'name': 'Pandhari', 'location': 'Parbhani', 'likes': ['strong coffee'], 'dislikes': None, 'allergies': ['nuts']}] count=None

Fetching...
{'id': 234, 'created_at': '2025-07-05T10:48:53.262985+00:00', 'name': 'Pandhari', 'location': 'Parbhani', 'likes': ['strong coffee'], 'dislikes': None, 'allergies': ['nuts']}

Updating allergies...
data=[{'id': 234, 'created_at': '2025-07-05T10:48:53.835204+00:00', 'name': 'Pandhari', 'location': 'Parbhani', 'likes': ['strong coffee'], 'dislikes': None, 'allergies': ['dairy', 'nuts']}] count=None


In [37]:
# Backend/memory/supabase_client.py
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv()
supabase = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))

In [ ]:
from typing import Optional
from datetime import datetime




def get_user_memory(user_id: str) -> UserMemory:
    res = supabase.table("User_Profiles").select("*").eq("id", user_id).execute()
    data = res.data[0] if res.data else {}
    return UserMemory(**data)


def save_user_memory(user_id: str, memory: UserMemory):
    payload = memory.dict()
    payload["id"] = user_id
    payload["last_updated"] = datetime.utcnow().isoformat()
    supabase.table("User_Profiles").upsert(payload).execute()

In [41]:
def merge_and_update_memory(user_id: str, updates: dict) -> UserMemory:
    existing = get_user_memory(user_id)

    for key, new_val in updates.items():
        if isinstance(new_val, list):
            current = getattr(existing, key, [])
            merged = list(set(current + new_val))
            setattr(existing, key, merged)
        else:
            setattr(existing, key, new_val)

    save_user_memory(user_id, existing)
    return existing

In [42]:
def remove_from_memory(user_id: str, to_remove: dict) -> UserMemory:
    existing = get_user_memory(user_id)

    for field, values in to_remove.items():
        if isinstance(values, list) and hasattr(existing, field):
            current = getattr(existing, field, [])
            filtered = [v for v in current if v.lower() not in map(str.lower, values)]
            setattr(existing, field, filtered)
        elif isinstance(values, str) and hasattr(existing, field):
            if getattr(existing, field) == values:
                setattr(existing, field, None)

    save_user_memory(user_id, existing)
    return existing

In [45]:
# Upsert
save_user_memory(12, UserMemory(name="Paddy", likes=["vanilla"]))

# Merge likes
merge_and_update_memory(12, {"likes": ["hazelnut"]})

# Remove dislikes
remove_from_memory(12, {"dislikes": ["too much milk"]})

/var/folders/42/94lj0cp519ncj7_m79j3_h3m0000gn/T/ipykernel_9000/2950491926.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  payload = memory.dict()


UserMemory(name='Paddy', likes=['vanilla', 'hazelnut'], dislikes=[], allergies=[], location=None)

In [47]:
merge_and_update_memory(12, {"name": "kd"})

/var/folders/42/94lj0cp519ncj7_m79j3_h3m0000gn/T/ipykernel_9000/2950491926.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  payload = memory.dict()


UserMemory(name='kd', likes=['vanilla', 'hazelnut'], dislikes=['Latte'], allergies=[], location=None)